In [ ]:
import os
import sys

# Clone or pull part
repo_url = "https://github.com/fraco03/6D_pose.git"
repo_dir = "/kaggle/working/6D_pose"   #Modify here for kaggle
branch = "main"

# Clone if missing
if not os.path.exists(repo_dir):
    !git clone -b {branch} {repo_url}
    print(f"Cloned {repo_url} to {repo_dir}")
else:
    %cd {repo_dir}
    !git fetch origin
    !git checkout {branch}
    !git reset --hard origin/{branch}
    %cd ..
    print(f"Updated {repo_url} to {repo_dir}")

# Add repository to Python path
if repo_dir not in sys.path:
    sys.path.insert(0, repo_dir)

Cloning into '6D_pose'...
remote: Enumerating objects: 364, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 364 (delta 10), reused 3 (delta 1), pack-reused 334 (from 1)
Receiving objects: 100% (364/364), 5.78 MiB | 23.04 MiB/s, done.
Resolving deltas: 100% (176/176), done.
Cloned https://github.com/fraco03/6D_pose.git to /content/6D_pose


In [ ]:
!pip install -U ultralytics
from ultralytics import YOLO
from src.detection.yolo_utils import calculate_adapted_map50, visualize_bbox, create_teacher_dataset_final, create_student_dataset_final

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 20.8 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
# CELL 3: GENERATE SYNTHETIC TRAINING DATASET WITH COLLAGES
import os
import cv2
import numpy as np
import random
import yaml
import shutil
import requests
from tqdm import tqdm

# --- CONFIGURATION ---
SOURCE_ROOT = '/kaggle/input/line-mode/Linemod_preprocessed/data'
DEST_ROOT = '/kaggle/working/datasets'
BG_CACHE_DIR = '/kaggle/working/backgrounds_cache'

NUM_COLLAGES = 4000      # Number of synthetic collage images to generate
MAX_OBJECTS_PER_IMG = 7  # Maximum objects per collage image

train_subset, test_subset = create_teacher_dataset_final(SOURCE_ROOT,DEST_ROOT,BG_CACHE_DIR,NUM_COLLAGES,MAX_OBJECTS_PER_IMG)

🛠️  Setting up directories...
🌍 Downloading background images...
♻️  Loading objects for collages (excluding class 02)...


100%|██████████| 13/13 [00:11<00:00,  1.10it/s]


🚀 Generating 2000 synthetic collages...


100%|██████████| 2000/2000 [00:17<00:00, 111.38it/s]


📸 Processing real images...


100%|██████████| 13/13 [01:50<00:00,  8.49s/it]


✅ Dataset generated successfully!
   [TRAINING]  Synthetic collages (no class 2):  2000
   [TRAINING]  Real images (class 2 only):       30
   [SKIPPED]   Real training images (others):    374
   [VALIDATION] Real images (all classes):       15396


In [ ]:
# CELL 4: TRAIN YOLO MODEL ON SYNTHETIC DATA
import torch

# --- 1. HARDWARE CHECK ---
if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
else:
    print("⚠️ WARNING: Using CPU! Go to Runtime > Change runtime type > T4 GPU")

# --- 2. MODEL SELECTION ---
# YOLO11 Small is perfect for T4 GPU
model = YOLO('yolo11s.pt')

print("🚀 Starting Training on Colab T4...")

# --- 3. TRAINING LOOP ---
results = model.train(
    data='/kaggle/working/linemod.yaml',

    # --- DURATION ---
    epochs=60,           # Increased to 100. Synthetic data requires more time to converge
    patience=15,         # Early stopping if no improvement for 15 epochs

    # --- HARDWARE ---
    imgsz=640,
    batch=32,            # Batch size 16 is safer and more stable for generalization on difficult datasets
    device=[0, 1],
    workers=4,
    amp=True,            # Mixed Precision ON (Essential for T4)

    # --- AUGMENTATION (Tuned for "Black-on-Black" scenarios) ---
    mosaic=0.5,          # REDUCED. Data is already collaged, don't overdo it
    mixup=0.0,           # DISABLED. Critical! We want solid objects, not transparent ghosts

    degrees=10.0,        # Light rotation (already done in generator script)
    scale=0.5,           # Zoom in/out
    translate=0.1,       # Light translation
    fliplr=0.5,          # Horizontal flip OK

    # --- LIGHTING (YOLO applies these instead of our script) ---
    hsv_h=0.015,         # Light color shift
    hsv_s=0.7,           # Saturation
    hsv_v=0.4,           # BRIGHTNESS (Value): 0.4 is perfect
                         # Teaches the model: "The phone is the same whether dark (black) or illuminated (gray)"

    close_mosaic=10,     # Last 10 epochs turn off mosaic to refine edges

    # --- SAVE RESULTS ---
    # WARNING: This saves directly to Google Drive
    project='/kaggle/working/YOLO_Runs',
    name='yolo11s_collage_run',
    verbose=True,
    exist_ok=True,
    save=True
)

✅ GPU: Tesla T4
🚀 Starting Training on Colab T4...
Ultralytics 8.3.240 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/linemod.yaml, degrees=10.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=0.5, multi_scale=False, name=yolo11_collage_run, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=T

KeyboardInterrupt: 

In [ ]:
import os
import random
import cv2
import matplotlib.pyplot as plt
from ultralytics import YOLO

# --- CONFIGURATION ---
MODEL_PATH = '/kaggle/working/YOLO_Runs/yolo11_collage_run/weights/best.pt'
TEST_DIR = '/kaggle/working/datasets/images/test'  # Directory containing images
CONF_THRESHOLD = 0.3  # Confidence threshold

# 1. Load Trained Model
print(f"🧠 Loading model from: {MODEL_PATH}")
model = YOLO(MODEL_PATH)

# 2. Select a Random Image from the Test Directory
# Get list of valid image files
image_files = [f for f in os.listdir(TEST_DIR) if f.endswith(('.png', '.jpg', '.jpeg'))]

if not image_files:
    raise FileNotFoundError(f"❌ No images found in directory: {TEST_DIR}")

# Pick one random file
random_filename = random.choice(image_files)
img_path = os.path.join(TEST_DIR, random_filename)
print(f"🎲 Selected Image: {random_filename}")

# 3. Define Class Names Mapping
# Mapping IDs to readable names
class_map = {
    0: 'ape', 1: 'benchvise', 2: 'camera', 3: 'can', 4: 'cat',
    5: 'driller', 6: 'duck', 7: 'eggbox', 8: 'glue', 9: 'holepuncher',
    10: 'iron', 11: 'lamp', 12: 'phone'
}

# 4. Run Inference on the selected image
results = model.predict(img_path, conf=CONF_THRESHOLD)

# 5. Visualize Results
# Load image for drawing
img = cv2.imread(img_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Check if detections exist
if len(results[0].boxes) == 0:
    print("⚠️ No objects detected in this image.")
else:
    # Iterate through detections
    for box in results[0].boxes:
        cls_id = int(box.cls[0])    # Class ID
        conf = float(box.conf[0])   # Confidence score
        
        # Get coordinates
        x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)
        
        # Get name (safely handle missing keys)
        label_name = class_map.get(cls_id, f"Unknown_{cls_id}")
        label_text = f"{label_name} {conf:.2f}"
        
        # Draw Rectangle
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
        
        # Draw Label Background
        (w, h), _ = cv2.getTextSize(label_text, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
        cv2.rectangle(img, (x1, y1 - 20), (x1 + w, y1), (0, 255, 0), -1)
        
        # Draw Text
        cv2.putText(img, label_text, (x1, y1 - 5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 2)

# Display final result
plt.figure(figsize=(10, 10))
plt.imshow(img)
plt.axis('off')
plt.title(f'Detection: {random_filename}')
plt.show()

In [ ]:
# CELL 6: AUTO-LABELING - USE TRAINED MODEL TO ADD MISSING LABELS
import os
import cv2
import numpy as np
import yaml
import shutil
from ultralytics import YOLO
from tqdm import tqdm


DEST_ROOT = '/kaggle/working/dataset_AUTOLABELED'
MODEL_PATH = '/kaggle/working/YOLO_Runs/yolo11s_collage_run/weights/best.pt'

create_student_dataset_final(DEST_ROOT, MODEL_PATH, train_subset,'/kaggle/working/datasets')

In [ ]:
# CELL 9: FINAL TRAINING ON AUTO-LABELED DATASET
from ultralytics import YOLO

# Will automatically download standard weights if not found
model = YOLO('yolo11m.pt')

results = model.train(
    # The clean dataset with automatic labels
    data='/kaggle/working/dataset_AUTOLABELED/data.yaml',

    # --- DURATION ---
    epochs=80,
    patience=15,

    # --- HARDWARE ---
    imgsz=640,
    batch=32,            # With the 's' model, use 16 to avoid running out of video memory (OOM)
    device=0,
    workers=4,
    cache=True,
    # --- AUGMENTATION ---
    mosaic=1.0,          # Now that labels are good, mosaic is useful
    mixup=0.1,           # Light mixup

    degrees=10.0,
    fliplr=0.5,
    scale=0.5,

    # --- OUTPUT ---
    project='/kaggle/working/YOLO_Runs',
    name='yolo11s_autolabel_final_with_70_th', # Changed name to remind it's the S model
    save=True,
    exist_ok=True
)

In [ ]:
calculate_adapted_map50('/content/drive/MyDrive/YOLO_Runs/yolo11s_autolabel_final/weights/best.pt',
                        '/content/data/Linemod_preprocessed/data')